In [4]:
import dspy as ds
import pandas as pd
import numpy as np
from rich import print
from dspy.datasets.gsm8k import GSM8K, gsm8k_metric
from tqdm import tqdm
from dspy.evaluate import Evaluate
from dspy.teleprompt import BootstrapFewShot
from dspy.datasets import HotPotQA
from dsp.utils import deduplicate

tqdm.pandas()

%load_ext rich


## GSM8K


In [5]:
turbo = ds.OpenAI(model="gpt-3.5-turbo-instruct", max_tokens=250)
ds.settings.configure(lm=turbo, log_openai_usage=True)

In [6]:
gsm8k = GSM8K()

train_gsm8k, dev_gsm8k = gsm8k.train[:10], gsm8k.dev[:10]

print(train_gsm8k)


100%|██████████| 1319/1319 [00:00<00:00, 21621.31it/s]


[
    Example({'question': "The result from the 40-item Statistics exam Marion and Ella took already came out. Ella 
got 4 incorrect answers while Marion got 6 more than half the score of Ella. What is Marion's score?", 
'gold_reasoning': "Ella's score is 40 items - 4 items = <<40-4=36>>36 items. Half of Ella's score is 36 items / 2 =
<<36/2=18>>18 items. So, Marion's score is 18 items + 6 items = <<18+6=24>>24 items.", 'answer': '24'}) 
(input_keys={'question'}),
    Example({'question': "Stephen made 10 round trips up and down a 40,000 foot tall mountain. If he reached 3/4 of
the mountain's height on each of his trips, calculate the total distance he covered.", 'gold_reasoning': 'Up a 
mountain, Stephen covered 3/4*40000 = <<3/4*40000=30000>>30000 feet. Coming down, Stephen covered another 30000 
feet, making the total distance covered in one round to be 30000+30000 = <<30000+30000=60000>>60000. Since Stephen 
made 10 round trips up and down the mountain, he covered 10*60000 = <<10*60000=600000>>600000', 'answer': 
'600000'}) (input_keys={'question'}),
    Example({'question': 'Bridget counted 14 shooting stars in the night sky.  Reginald counted two fewer shooting 
stars than did Bridget, but Sam counted four more shooting stars than did Reginald.  How many more shooting stars 
did Sam count in the night sky than was the average number of shooting stars observed for the three of them?', 
'gold_reasoning': 'Reginald counted two fewer shooting stars than did Bridget, or a total of 14-2=<<14-2=12>>12 
shooting stars. Sam counted 4 more shooting stars than did Reginald, or a total of 12+4=16 shooting stars. The 
average number of shooting stars observed for the three of them was (14+12+16)/3 = <<14=14>>14 shooting stars. 
Thus, Sam counted 16-14=2 more shooting stars than was the average number of shooting stars observed for the three 
of them.', 'answer': '2'}) (input_keys={'question'}),
    Example({'question': 'Sarah buys 20 pencils on Monday. Then she buys 18 more pencils on Tuesday. On Wednesday 
she buys triple the number of pencils she did on Tuesday. How many pencils does she have?', 'gold_reasoning': 'By 
adding together Monday and Tuesday, Saah has 20+18= <<20+18=38>>38 pencils On Wednesday, she buys 3 * 18= 
<<3*18=54>>54 pencils All together, Sarah has 38+54= <<38+54=92>>92 pencils', 'answer': '92'}) 
(input_keys={'question'}),
    Example({'question': 'Rookie police officers have to buy duty shoes at the full price of $85, but officers who 
have served at least a year get a 20% discount. Officers who have served at least three years get an additional 25%
off the discounted price. How much does an officer who has served at least three years have to pay for shoes?', 
'gold_reasoning': 'Cops that served a year pay $85 * 0.2 = $<<85*0.2=17>>17 less. Cops that served a year pay $85 -
$17 = $<<85-17=68>>68. Cops that served at least 3 years get a $68 * 0.25 = $<<68*0.25=17>>17 discount. Cops that 
served at least 3 years pay $68 - $17 = $<<68-17=51>>51 for shoes.', 'answer': '51'}) (input_keys={'question'}),
    Example({'question': "The average score on last week's Spanish test was 90.  Marco scored 10% less than the 
average test score and Margaret received 5 more points than Marco.  What score did Margaret receive on her test?", 
'gold_reasoning': 'The average test score was 90 and Marco scored 10% less so 90*.10 = <<90*.10=9>>9 points lower 
The average test score was 90 and Marco scored 9 points less so his test score was 90-9 = <<90-9=81>>81 Margret 
received 5 more points than Marco whose test score was 81 so she made 5+81 = <<5+81=86>>86 on her test', 'answer': 
'86'}) (input_keys={'question'}),
    Example({'question': 'A third of the contestants at a singing competition are female, and the rest are male. If
there are 18 contestants in total, how many of them are male?', 'gold_reasoning': 'There are 18/3 = <<18/3=6>>6 
female contestants. There are 18-6 = <<18-6=12>>12 male contestants.', 'answer': '12'}) (input

In [7]:
print(train_gsm8k[0])

Example({'question': "The result from the 40-item Statistics exam Marion and Ella took already came out. Ella got 4
incorrect answers while Marion got 6 more than half the score of Ella. What is Marion's score?", 'gold_reasoning': 
"Ella's score is 40 items - 4 items = <<40-4=36>>36 items. Half of Ella's score is 36 items / 2 = <<36/2=18>>18 
items. So, Marion's score is 18 items + 6 items = <<18+6=24>>24 items.", 'answer': '24'}) (input_keys={'question'})

### Define the modules


In [8]:
class CoT(ds.Module):
    def __init__(self):
        super().__init__()
        self.prog = ds.ChainOfThought("question -> answer")

    def forward(self, question):
        return self.prog(question=question)

In [9]:
teleprompter = BootstrapFewShot(
    metric=gsm8k_metric, max_bootstrapped_demos=4, max_labeled_demos=4
)

optimized_cot = teleprompter.compile(CoT(), trainset=train_gsm8k, valset=dev_gsm8k)


 50%|█████     | 5/10 [00:00<00:00, 37.59it/s]


Bootstrapped 4 full traces after 6 examples in round 0.


In [10]:
optimized_cot(question="What is the capital of France?")


Prediction(
    rationale='find the answer. First, we know that the country is France. Then, we know that the capital is the most important city in a country. Therefore, the capital of France is Paris.',
    answer='Paris'
)

In [11]:
evalute = Evaluate(
    devset=dev_gsm8k,
    metric=gsm8k_metric,
    num_threads=4,
    display_progress=True,
    display_table=1,
)

In [12]:
evalute(optimized_cot)

Average Metric: 9 / 10  (90.0): 100%|██████████| 10/10 [00:00<00:00, 104.17it/s]


Average Metric: 9 / 10  (90.0%)


e:\Projects\langchain_rag\venv\lib\site-packages\dspy\evaluate\evaluate.py:266: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['✔️ [True]']' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[:, metric_name] = df[metric_name].apply(


,question,gold_reasoning,example_answer,rationale,pred_answer,gsm8k_metric
0,"20 birds migrate on a seasonal basis from one lake to another, searching for food. If they fly from lake Jim to lake Disney in...",The birds' flying distance between Lake Jim through lake Disney to lake London is 50+60 = <<50+60=110>>110 miles. Since each bird flies the 110 miles...,2200,"calculate the combined distance traveled by the birds. First, we know that the birds flew 50 miles from lake Jim to lake Disney and then...",2200 miles,✔️ [True]


<IPython.core.display.HTML object>

90.0

In [13]:
turbo.inspect_history()




Given the fields `question`, produce the fields `answer`.

---

Follow the following format.

Question: ${question}
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: ${answer}

---

Question: The result from the 40-item Statistics exam Marion and Ella took already came out. Ella got 4 incorrect answers while Marion got 6 more than half the score of Ella. What is Marion's score?
Reasoning: Let's think step by step in order to find Marion's score. We know that Ella got 4 incorrect answers, which means she got 36 correct answers out of 40. We also know that Marion got 6 more than half of Ella's score, which is 6 more than 36/2 = 18. Therefore, Marion's score is 18 + 6 = 24.
Answer: 24

---

Question: Bridget counted 14 shooting stars in the night sky. Reginald counted two fewer shooting stars than did Bridget, but Sam counted four more shooting stars than did Reginald. How many more shooting stars did Sam count in the night sky than was the average n

"\n\n\nGiven the fields `question`, produce the fields `answer`.\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\nReasoning: Let's think step by step in order to ${produce the answer}. We ...\nAnswer: ${answer}\n\n---\n\nQuestion: The result from the 40-item Statistics exam Marion and Ella took already came out. Ella got 4 incorrect answers while Marion got 6 more than half the score of Ella. What is Marion's score?\nReasoning: Let's think step by step in order to find Marion's score. We know that Ella got 4 incorrect answers, which means she got 36 correct answers out of 40. We also know that Marion got 6 more than half of Ella's score, which is 6 more than 36/2 = 18. Therefore, Marion's score is 18 + 6 = 24.\nAnswer: 24\n\n---\n\nQuestion: Bridget counted 14 shooting stars in the night sky. Reginald counted two fewer shooting stars than did Bridget, but Sam counted four more shooting stars than did Reginald. How many more shooting stars did Sam count in the night sky 

## With retrieval module


In [14]:
turbo = ds.OpenAI(model="gpt-3.5-turbo")
colbertv2_wiki17_abstracts = ds.ColBERTv2(
    url="http://20.102.90.50:2017/wiki17_abstracts"
)

ds.settings.configure(lm=turbo, rm=colbertv2_wiki17_abstracts, log_openai_usage=True)


In [15]:
dataset_hpqa = HotPotQA(
    train_seed=1, train_size=20, eval_seed=2023, dev_size=50, test_size=0
)

e:\Projects\langchain_rag\venv\lib\site-packages\datasets\table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [16]:
trainset_hpqa = [x.with_inputs("question") for x in dataset_hpqa.train]
devset_hpqa = [x.with_inputs("question") for x in dataset_hpqa.dev]

len(trainset_hpqa), len(devset_hpqa)


(20, 50)

In [17]:
print(trainset_hpqa[2])

Example({'question': 'Which of these publications was most recently published, Who Put the Bomp or Self?', 
'answer': 'Self'}) (input_keys={'question'})

In [18]:
print(devset_hpqa[0])

Example({'question': 'Are both Cangzhou and Qionghai in the Hebei province of China?', 'answer': 'no', 
'gold_titles': {'Cangzhou', 'Qionghai'}}) (input_keys={'question'})

### Signature

In [19]:
class BasicQA(ds.Signature):
    """Answer questions with short factoid layers"""

    question = ds.InputField()
    answer = ds.OutputField(desc='often between 1 and 5 words')

In [20]:
BasicQA


BasicQA(question -> answer
    instructions='Answer questions with short factoid layers'
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'})
    answer = Field(annotation=str required=True json_schema_extra={'desc': 'often between 1 and 5 words', '__dspy_field_type': 'output', 'prefix': 'Answer:'})
)

In [21]:
generate_answer = ds.Predict(BasicQA)

In [22]:
pred = generate_answer(question=devset_hpqa[3].question)

In [23]:
print(f'Question: {devset_hpqa[3].question}')
print(f'Answer: {pred.answer}')

Question: What river is near the Crichton Collegiate Church?

Answer: River Nith

In [24]:
turbo.inspect_history(n=3)




Answer questions with short factoid layers

---

Follow the following format.

Question: ${question}
Answer: often between 1 and 5 words

---

Question: What river is near the Crichton Collegiate Church?
Answer: River Nith





'\n\n\nAnswer questions with short factoid layers\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\nAnswer: often between 1 and 5 words\n\n---\n\nQuestion: What river is near the Crichton Collegiate Church?\nAnswer:\x1b[32m River Nith\x1b[0m\n\n\n'

In [25]:
generate_cot = ds.ChainOfThought(BasicQA)

pred = generate_cot(question=devset_hpqa[14].question)

print(f'Question: {devset_hpqa[14].question}')
print(f'Thought: {pred.rationale.split(".", 1)[1].strip()}')
print(f'Answer: {pred.answer}')


Question: Lord North Street has a resident in which former Conservative MP who received an 18-month prison sentence
for perjury in 1999?

Thought: We know that Lord North Street is a prestigious address in London, and the former Conservative MP who 
received an 18-month prison sentence for perjury in 1999 is Jonathan Aitken.

Answer: Jonathan Aitken

In [26]:
retrieve = ds.Retrieve(k=3)

top_k_passages = retrieve(devset_hpqa[14].question).passages

print(
    f"Top {retrieve.k} passages for question: {devset_hpqa[14].question} \n",
    "-" * 30,
    "\n",
)

for idx, passage in enumerate(top_k_passages):
    print(f"{idx+1}]", passage, "\n")


Top 3 passages for question: Lord North Street has a resident in which former Conservative MP who received an 
18-month prison sentence for perjury in 1999? 
 ------------------------------

1] Jonathan Aitken | Jonathan William Patrick Aitken (born 30 August 1942) is a former Conservative Member of 
Parliament in the United Kingdom (1974–97), and a former Cabinet minister. He was convicted of perjury in 1999 and 
received an 18-month prison sentence, of which he served seven months. He is currently president of Christian 
Solidarity Worldwide. Aitken was also a member of the Privy Council of the United Kingdom.

2] David Chaytor | David Michael Chaytor (born 3 August 1949) is a former British Labour Party politician, who was 
the Member of Parliament (MP) for Bury North from 1997 to 2010. He was the first member of Parliament to be 
sentenced following the United Kingdom Parliamentary expenses scandal of 2009.

3] Elliot Morley | Elliot Anthony Morley (born 6 July 1952) is a British former Labour Party politician, who was 
the Member of Parliament for Glanford and Scunthorpe from 1987 to 1997 and then Scunthorpe from 1997 to 2010. In 
2009, he was accused by "The Daily Telegraph" of continuing to claim parliamentary expenses for a mortgage that had
already been repaid. Morley was prosecuted and on 7 April 2011 pleaded guilty in Southwark Crown Court to two 
counts of false accounting, involving over £30,000. On 20 May 2011, he was sentenced to 16 months' imprisonment. He
was released from prison on 20 September 2011 having served a quarter of his sentence.

In [27]:
retrieve("Who is the president of the United States?").passages


[
    'President of the United States (disambiguation) | The President of the United States (of America) has been chief of the United States executive branch since 1789. The current U.S. President is Donald Trump.',
    'President of the United States | The President of the United States (informally referred to as "POTUS") is the head of state and head of government of the United States. The president directs the executive branch of the federal government and is the commander-in-chief of the United States Armed Forces.',
    'List of Presidents of the United States | The President of the United States is the elected head of state and head of government of the United States. The president leads the executive branch of the federal government and is the commander-in-chief of the United States Armed Forces. The president is indirectly elected to a four-year term by the people through the Electoral College (or by the House of Representatives, should the Electoral College fail to award an a

## Program 1: Basic RAG

In [28]:
class GenerateAnswer(ds.Signature):
    """Answer question with short factoid layers"""

    question = ds.InputField()
    context = ds.InputField(desc="may contain relevant facts")
    answer = ds.OutputField(desc="often between 1 and 5 words")

In [29]:
class RAG(ds.Module):
    def __init__(self, num_passages=3):
        super().__init__()
        
        self.retrieve = ds.Retrieve(k=num_passages)
        self.generate_answer = ds.ChainOfThought(GenerateAnswer)

    def forward(self, question):
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)

        return ds.Prediction(context=context, answer=prediction.answer)

In [30]:
def validate_context_and_answer(example, pred, trace=None):
    answer_EM = ds.evaluate.answer_exact_match(example, pred)
    answer_PM = ds.evaluate.answer_passage_match(example, pred)

    return answer_EM and answer_PM

In [31]:
teleprompter = BootstrapFewShot(metric=validate_context_and_answer)
compiled_rag = teleprompter.compile(RAG(), trainset=trainset_hpqa)

 55%|█████▌    | 11/20 [00:00<00:00, 139.23it/s]


Bootstrapped 4 full traces after 12 examples in round 0.


In [32]:
compiled_rag


generate_answer = ChainOfThought(GenerateAnswer(question, context -> answer
    instructions='Answer question with short factoid layers'
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'})
    context = Field(annotation=str required=True json_schema_extra={'desc': 'may contain relevant facts', '__dspy_field_type': 'input', 'prefix': 'Context:'})
    answer = Field(annotation=str required=True json_schema_extra={'desc': 'often between 1 and 5 words', '__dspy_field_type': 'output', 'prefix': 'Answer:'})
))

In [33]:
question = "Who is the current president of the United States?"

pred = compiled_rag(question)

print(f"Question: {question}")
print(f'Context: {[c[:200] + "..." for c in pred.context]}')
print(f"Answer: {pred.answer}")

Question: Who is the current president of the United States?

Context: ['President of the United States (disambiguation) | The President of the United States (of America) has 
been chief of the United States executive branch since 1789. The current U.S. President is Donald ...', 'Donald 
Trump | Donald John Trump (born June 14, 1946) is the 45th and current President of the United States, in office 
since January 20, 2017. Before entering politics, he was a businessman and telev...', 'President of Mexico | The 
President of the United Mexican States (Spanish: "Presidente de los Estados Unidos Mexicanos" ) is the head of 
state and government of Mexico. Under the Constitution, the pre...']

Answer: Donald Trump

In [34]:
turbo.inspect_history(n=1)




Answer question with short factoid layers

---

Question: At My Window was released by which American singer-songwriter?
Answer: John Townes Van Zandt

Question: "Everything Has Changed" is a song from an album released under which record label ?
Answer: Big Machine Records

Question: The Victorians - Their Story In Pictures is a documentary series written by an author born in what year?
Answer: 1950

Question: Which Pakistani cricket umpire who won 3 consecutive ICC umpire of the year awards in 2009, 2010, and 2011 will be in the ICC World Twenty20?
Answer: Aleem Sarwar Dar

Question: Having the combination of excellent foot speed and bat speed helped Eric Davis, create what kind of outfield for the Los Angeles Dodgers?
Answer: "Outfield of Dreams"

Question: Who is older, Aleksandr Danilovich Aleksandrov or Anatoly Fomenko?
Answer: Aleksandr Danilovich Aleksandrov

Question: The Organisation that allows a community to influence their operation or use and to enjoy the benefits aris

'\n\n\nAnswer question with short factoid layers\n\n---\n\nQuestion: At My Window was released by which American singer-songwriter?\nAnswer: John Townes Van Zandt\n\nQuestion: "Everything Has Changed" is a song from an album released under which record label ?\nAnswer: Big Machine Records\n\nQuestion: The Victorians - Their Story In Pictures is a documentary series written by an author born in what year?\nAnswer: 1950\n\nQuestion: Which Pakistani cricket umpire who won 3 consecutive ICC umpire of the year awards in 2009, 2010, and 2011 will be in the ICC World Twenty20?\nAnswer: Aleem Sarwar Dar\n\nQuestion: Having the combination of excellent foot speed and bat speed helped Eric Davis, create what kind of outfield for the Los Angeles Dodgers?\nAnswer: "Outfield of Dreams"\n\nQuestion: Who is older, Aleksandr Danilovich Aleksandrov or Anatoly Fomenko?\nAnswer: Aleksandr Danilovich Aleksandrov\n\nQuestion: The Organisation that allows a community to influence their operation or use and 

In [35]:
for name, parameter in compiled_rag.named_predictors():
    print(name)
    print(parameter.demos[0])
    print()

generate_answer

Example({'augmented': True, 'context': ['Tae Kwon Do Times | Tae Kwon Do Times is a magazine devoted to the martial
art of taekwondo, and is published in the United States of America. While the title suggests that it focuses on 
taekwondo exclusively, the magazine also covers other Korean martial arts. "Tae Kwon Do Times" has published 
articles by a wide range of authors, including He-Young Kimm, Thomas Kurz, Scott Shaw, and Mark Van Schuyver.', 
"Kwon Tae-man | Kwon Tae-man (born 1941) was an early Korean hapkido practitioner and a pioneer of the art, first 
in Korea and then in the United States. He formed one of the earliest dojang's for hapkido in the United States in 
Torrance, California, and has been featured in many magazine articles promoting the art.", 'Hee Il Cho | Cho Hee Il
(born October 13, 1940) is a prominent Korean-American master of taekwondo, holding the rank of 9th "dan" in the 
martial art. He has written 11 martial art books, produced 70 martial art training videos, and has appeared on more
than 70 martial arts magazine covers. Cho won several national and international competitions as a taekwondo 
competitor, and has appeared in several films, including "Fight to Win", "Best of the Best", "Bloodsport II", and 
"Bloodsport III". He founded the Action International Martial Arts Association (AIMAA) in 1980, and is its 
President. Cho is a member of both "Black Belt" magazine\'s Hall of Fame and "Tae Kwon Do Times" magazine\'s Hall 
of Fame.'], 'question': 'Which magazine has published articles by Scott Shaw, Tae Kwon Do Times or Southwest Art?',
'rationale': 'determine which magazine has published articles by Scott Shaw. We know that "Tae Kwon Do Times" has 
published articles by Scott Shaw, so the answer is:', 'answer': 'Tae Kwon Do Times'}) (input_keys=None)

In [36]:
evaluate_on_hotpotqa = Evaluate(
    devset=devset_hpqa,
    num_threads=4,
    display_progress=True,
    display_table=5,
    metric=ds.evaluate.answer_exact_match,
)

evaluate_on_hotpotqa(compiled_rag)

Average Metric: 27 / 43  (62.8):  84%|████████▍ | 42/50 [00:00<00:00, 61.58it/s]

Average Metric: 28 / 50  (56.0): 100%|██████████| 50/50 [00:00<00:00, 59.74it/s]


Average Metric: 28 / 50  (56.0%)


e:\Projects\langchain_rag\venv\lib\site-packages\dspy\evaluate\evaluate.py:266: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['✔️ [True]' '✔️ [True]' '✔️ [True]' '✔️ [True]' '✔️ [True]']' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[:, metric_name] = df[metric_name].apply(


,question,example_answer,gold_titles,context,pred_answer,answer_exact_match
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{'Cangzhou', 'Qionghai'}","['Cangzhou | Cangzhou () is a prefecture-level city in eastern Hebei province, People\'s Republic of China. At the 2010 census, Cangzhou\'s built-up (""or metro"") area...",No,✔️ [True]
1,Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?,National Hockey League,"{'2017–18 Pittsburgh Penguins season', '2017 NHL Expansion Draft'}",['2017–18 Pittsburgh Penguins season | The 2017–18 Pittsburgh Penguins season will be the 51st season for the National Hockey League ice hockey team that was...,National Hockey League,✔️ [True]
2,"The Wings entered a new era, following the retirement of which Canadian retired professional ice hockey player and current general manager of the Tampa Bay...",Steve Yzerman,"{'2006–07 Detroit Red Wings season', 'Steve Yzerman'}","['Steve Yzerman | Stephen Gregory ""Steve"" Yzerman ( ; born May 9, 1965) is a Canadian retired professional ice hockey player and current general manager...",Steve Yzerman,✔️ [True]
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{'Crichton Collegiate Church', 'Crichton Castle'}","[""Crichton Collegiate Church | Crichton Collegiate Church is situated about 0.6 mi south west of the hamlet of Crichton in Midlothian, Scotland. Crichton itself is...",River Tyne,✔️ [True]
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?,King Alfred the Great,"{'Ealhswith', 'Æthelweard (son of Alfred)'}","[""Æthelweard of East Anglia | Æthelweard (died 854) was a 9th-century king of East Anglia, the long-lived Anglo-Saxon kingdom which today includes the English counties...",King Alfred the Great,✔️ [True]


<IPython.core.display.HTML object>

56.0

In [37]:
def gold_passages_retrieved(example, pred, trace=None):
    gold_titles = set(map(ds.evaluate.normalize_text, example['gold_titles']))
    found_titles = set(map(ds.evaluate.normalize_text, [c.split("|")[0] for c in pred.context]))

    return gold_titles.issubset(found_titles)

compiled_rag_retrieval_score = evaluate_on_hotpotqa(compiled_rag, metric=gold_passages_retrieved)

Average Metric: 13 / 50  (26.0): 100%|██████████| 50/50 [00:00<00:00, 276.56it/s]


Average Metric: 13 / 50  (26.0%)


e:\Projects\langchain_rag\venv\lib\site-packages\dspy\evaluate\evaluate.py:266: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['False' '✔️ [True]' '✔️ [True]' '✔️ [True]' 'False']' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[:, metric_name] = df[metric_name].apply(


,question,example_answer,gold_titles,context,pred_answer,gold_passages_retrieved
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{'Cangzhou', 'Qionghai'}","['Cangzhou | Cangzhou () is a prefecture-level city in eastern Hebei province, People\'s Republic of China. At the 2010 census, Cangzhou\'s built-up (""or metro"") area...",No,False
1,Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?,National Hockey League,"{'2017–18 Pittsburgh Penguins season', '2017 NHL Expansion Draft'}",['2017–18 Pittsburgh Penguins season | The 2017–18 Pittsburgh Penguins season will be the 51st season for the National Hockey League ice hockey team that was...,National Hockey League,✔️ [True]
2,"The Wings entered a new era, following the retirement of which Canadian retired professional ice hockey player and current general manager of the Tampa Bay...",Steve Yzerman,"{'2006–07 Detroit Red Wings season', 'Steve Yzerman'}","['Steve Yzerman | Stephen Gregory ""Steve"" Yzerman ( ; born May 9, 1965) is a Canadian retired professional ice hockey player and current general manager...",Steve Yzerman,✔️ [True]
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{'Crichton Collegiate Church', 'Crichton Castle'}","[""Crichton Collegiate Church | Crichton Collegiate Church is situated about 0.6 mi south west of the hamlet of Crichton in Midlothian, Scotland. Crichton itself is...",River Tyne,✔️ [True]
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?,King Alfred the Great,"{'Ealhswith', 'Æthelweard (son of Alfred)'}","[""Æthelweard of East Anglia | Æthelweard (died 854) was a 9th-century king of East Anglia, the long-lived Anglo-Saxon kingdom which today includes the English counties...",King Alfred the Great,False


<IPython.core.display.HTML object>

## Program 2: Multi-hop

In [38]:
turbo = ds.OpenAI(model="gpt-3.5-turbo")
colbertv2_wiki17_abstracts = ds.ColBERTv2(
    url="http://20.102.90.50:2017/wiki17_abstracts"
)

ds.settings.configure(lm=turbo, rm=colbertv2_wiki17_abstracts, log_openai_usage=True)

In [47]:
class GenerateSearchQuery(ds.Signature):
    """Write a simple search query that will help answer a complex question."""

    context = ds.InputField(desc="may contain relevant facts")
    question = ds.InputField()
    search_query = ds.OutputField(desc="search query to retrieve relevant facts")


In [48]:
class SimplifiedMultiHop(ds.Module):
    def __init__(self, passages_per_hop=3, max_hops=2):
        super().__init__()

        self.generate_query = [ds.ChainOfThought(GenerateSearchQuery) for _ in range(max_hops)]
        self.retrieve = ds.Retrieve(k=passages_per_hop)

        self.generate_answer = ds.ChainOfThought(GenerateAnswer)
        self.max_hops = max_hops

    def forward(self, question):
        context = []

        for hop in range(self.max_hops):
            query = self.generate_query[hop](context=context, question=question).search_query
            passages = self.retrieve(query).passages
            context = deduplicate(context + passages)

        prediction = self.generate_answer(context=context, question=question)

        return ds.Prediction(context=context, answer=prediction.answer)

In [49]:
simple_multi_hop = SimplifiedMultiHop()
question = "How many storeys are in the castle that David Gregory inherited?"

prediction = simple_multi_hop(question=question)

print(f"Question: {question}")
print(f"Answer: {prediction.answer}")
print(f"Context: {[c[:200] + '...' for c in prediction.context]}")


Question: How many storeys are in the castle that David Gregory inherited?

Answer: five

Context: ['David Gregory (physician) | David Gregory (20 December 1625 – 1720) was a Scottish physician and 
inventor. His surname is sometimes spelt as Gregorie, the original Scottish spelling. He inherited Kinn...', 'David
Gregory (footballer, born 1970) | Born in Polstead, Gregory began his career at Ipswich Town, making 32 appearances
between 1987–1995. He made two appearances on loan at Hereford United and thre...', 'Roughan Castle | Roughan 
Castle is a castle a mile outside Newmills, County Tyrone, Northern Ireland, on the Dungannon to Stewartstown road.
It was built about 1618 by Sir Andrew Stewart (d.1639), 2nd...', 'Kinnairdy Castle | Kinnairdy Castle is a tower 
house, having five storeys and a garret, two miles south of Aberchirder, Aberdeenshire, Scotland. The alternative 
name is Old Kinnairdy....', 'Kinnaird Castle, Brechin | Kinnaird Castle is a 15th-century castle in Angus, 
Scotland. The castle has been home to the Carnegie family, the Earl of Southesk, for more than 600 years....', 
'Kinnaird Head | Kinnaird Head (Scottish Gaelic: "An Ceann Àrd" , "high headland") is a headland projecting into 
the North Sea, within the town of Fraserburgh, Aberdeenshire on the east coast of Scotla...']

In [50]:
turbo.inspect_history(n=3)




Answer question with short factoid layers

---

Follow the following format.

Question: ${question}

Context: may contain relevant facts

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: often between 1 and 5 words

---

Question: How many storeys are in the castle that David Gregory inherited?

Context:
[1] «David Gregory (physician) | David Gregory (20 December 1625 – 1720) was a Scottish physician and inventor. His surname is sometimes spelt as Gregorie, the original Scottish spelling. He inherited Kinnairdy Castle in 1664. Three of his twenty-nine children became mathematics professors. He is credited with inventing a military cannon that Isaac Newton described as "being destructive to the human species". Copies and details of the model no longer exist. Gregory's use of a barometer to predict farming-related weather conditions led him to be accused of witchcraft by Presbyterian ministers from Aberdeen, although he was never convicted.»
[2] «

'\n\n\nAnswer question with short factoid layers\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\n\nContext: may contain relevant facts\n\nReasoning: Let\'s think step by step in order to ${produce the answer}. We ...\n\nAnswer: often between 1 and 5 words\n\n---\n\nQuestion: How many storeys are in the castle that David Gregory inherited?\n\nContext:\n[1] «David Gregory (physician) | David Gregory (20 December 1625 – 1720) was a Scottish physician and inventor. His surname is sometimes spelt as Gregorie, the original Scottish spelling. He inherited Kinnairdy Castle in 1664. Three of his twenty-nine children became mathematics professors. He is credited with inventing a military cannon that Isaac Newton described as "being destructive to the human species". Copies and details of the model no longer exist. Gregory\'s use of a barometer to predict farming-related weather conditions led him to be accused of witchcraft by Presbyterian ministers from Aberdeen, although he wa

In [51]:
def validate_context_and_answer_and_hops(example, pred, trace=None):
    if not ds.evaluate.answer_exact_match(example, pred):
        return False
    if not ds.evaluate.answer_passage_match(example, pred):
        return False

    hops = [example.question] + [
        outputs.query for *_, outputs in trace if "query" in outputs
    ]

    if max([len(h) for h in hops]) > 100:
        return False
    if any(
        ds.evaluate.answer_exact_match_str(hops[idx], hops[:idx], frac=0.8)
        for idx in range(2, len(hops))
    ):
        return False

    return True


In [52]:
teleprompter = BootstrapFewShot(metric=validate_context_and_answer_and_hops)
compiled_baleen = teleprompter.compile(
    SimplifiedMultiHop(),
    teacher=SimplifiedMultiHop(passages_per_hop=2),
    trainset=trainset_hpqa,
)


 35%|███▌      | 7/20 [00:28<00:52,  4.02s/it]


Bootstrapped 4 full traces after 8 examples in round 0.


In [53]:
uncompiled_evaluate = evaluate_on_hotpotqa(simple_multi_hop, metric=gold_passages_retrieved)
compiled_evaluate = evaluate_on_hotpotqa(compiled_baleen, metric=gold_passages_retrieved)

Average Metric: 27 / 50  (54.0): 100%|██████████| 50/50 [00:50<00:00,  1.02s/it]


Average Metric: 27 / 50  (54.0%)


e:\Projects\langchain_rag\venv\lib\site-packages\dspy\evaluate\evaluate.py:266: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['✔️ [True]' 'False' 'False' '✔️ [True]' 'False']' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[:, metric_name] = df[metric_name].apply(


,question,example_answer,gold_titles,context,pred_answer,gold_passages_retrieved
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{'Cangzhou', 'Qionghai'}","['Cangzhou | Cangzhou () is a prefecture-level city in eastern Hebei province, People\'s Republic of China. At the 2010 census, Cangzhou\'s built-up (""or metro"") area...",No.,✔️ [True]
1,Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?,National Hockey League,"{'2017–18 Pittsburgh Penguins season', '2017 NHL Expansion Draft'}","['2017 NHL Entry Draft | The 2017 NHL Entry Draft was the 55th NHL Entry Draft. The draft was held from June 23–24, 2017, at...",NHL Expansion Draft,False
2,"The Wings entered a new era, following the retirement of which Canadian retired professional ice hockey player and current general manager of the Tampa Bay...",Steve Yzerman,"{'2006–07 Detroit Red Wings season', 'Steve Yzerman'}","['Steve Yzerman | Stephen Gregory ""Steve"" Yzerman ( ; born May 9, 1965) is a Canadian retired professional ice hockey player and current general manager...",Steve Yzerman,False
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{'Crichton Collegiate Church', 'Crichton Castle'}","[""Crichton Collegiate Church | Crichton Collegiate Church is situated about 0.6 mi south west of the hamlet of Crichton in Midlothian, Scotland. Crichton itself is...",River Tyne,✔️ [True]
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?,King Alfred the Great,"{'Ealhswith', 'Æthelweard (son of Alfred)'}","['Æthelweard (son of Alfred) | Æthelweard (d. 920 or 922) was the younger son of King Alfred the Great and Ealhswith.', 'Æthelweard, king of the...",Alfred the Great,False


<IPython.core.display.HTML object>

Average Metric: 27 / 50  (54.0): 100%|██████████| 50/50 [00:59<00:00,  1.20s/it]


Average Metric: 27 / 50  (54.0%)


e:\Projects\langchain_rag\venv\lib\site-packages\dspy\evaluate\evaluate.py:266: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['✔️ [True]' '✔️ [True]' 'False' '✔️ [True]' 'False']' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[:, metric_name] = df[metric_name].apply(


,question,example_answer,gold_titles,context,pred_answer,gold_passages_retrieved
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{'Cangzhou', 'Qionghai'}","['Cangzhou | Cangzhou () is a prefecture-level city in eastern Hebei province, People\'s Republic of China. At the 2010 census, Cangzhou\'s built-up (""or metro"") area...",No,✔️ [True]
1,Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?,National Hockey League,"{'2017–18 Pittsburgh Penguins season', '2017 NHL Expansion Draft'}","[""Marc-André Fleury | Marc-André Fleury (born November 28, 1984) is a French-Canadian professional ice hockey goaltender playing for the Vegas Golden Knights of the National...",National Hockey League,✔️ [True]
2,"The Wings entered a new era, following the retirement of which Canadian retired professional ice hockey player and current general manager of the Tampa Bay...",Steve Yzerman,"{'2006–07 Detroit Red Wings season', 'Steve Yzerman'}","['Steve Yzerman | Stephen Gregory ""Steve"" Yzerman ( ; born May 9, 1965) is a Canadian retired professional ice hockey player and current general manager...",Steve Yzerman,False
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{'Crichton Collegiate Church', 'Crichton Castle'}","[""Crichton Collegiate Church | Crichton Collegiate Church is situated about 0.6 mi south west of the hamlet of Crichton in Midlothian, Scotland. Crichton itself is...",River Tyne,✔️ [True]
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?,King Alfred the Great,"{'Ealhswith', 'Æthelweard (son of Alfred)'}","['Æthelweard (son of Alfred) | Æthelweard (d. 920 or 922) was the younger son of King Alfred the Great and Ealhswith.', 'Æthelred the Unready |...",Alfred the Great,False


<IPython.core.display.HTML object>

In [54]:
print(f'Uncompiled model retrieval score: {uncompiled_evaluate}')
print(f'Compiled model retrieval score: {compiled_evaluate}')

Uncompiled model retrieval score: 54.0

Compiled model retrieval score: 54.0